In [106]:
#%pip install scikit-learn
#%pip install nltk
#%pip install utils
#%pip install emoji
#%pip install unidecode

In [107]:
import pandas as pd
import re
#from sklarn.feature_extraction.text import CountVectorizer
from gensim.parsing.preprocessing import STOPWORDS, strip_tags, strip_numeric, strip_punctuation, strip_multiple_whitespaces, remove_stopwords, strip_short, stem_text
import numpy as np
import sys
import os
sys.path.append('../')
#from utils import remove_similar_rows, find_lines_with_player, remove_similar_rows_per_player, remove_stopwords_from_text
#from utils import find_lines_with_player, map_emoji_to_description, remove_similar_rows_per_player ##del_patterns, 


#INFO included in utils - but utils doesn't work for me here
import emoji
def map_emoji_to_description(emoji_text, language): 
    emoji_description = emoji.demojize(emoji_text, language=language)
    return emoji_description

def translate_emojis(text, language):
    return re.sub(r'[\U0001F000-\U0001F999]', lambda match: map_emoji_to_description(match.group(), language=language), text)

def remove_stopwords_from_text(text, stopwords_list_per_language):
    return remove_stopwords(text, stopwords=stopwords_list_per_language)

from difflib import SequenceMatcher
def remove_similar_rows_per_player(df, playerlist, threshold=0.9):
    '''The procedure of deleting similiar articles needs to be done by each player because if an article writes about 
    # e.g. two players we want to keep it for both of the players'''

    # define empty df which will be returned in the end
    df_complete = pd.DataFrame()

    for player in playerlist:
        
        # create the df for the player
        df_player = df[df["player"] == player]
        df_player = df_player.reset_index(drop=True)
        column_as_df = pd.DataFrame(df_player['data'])


        
        # Compute similarity scores for each pair of rows
        similarity_scores = {}
        for i, row in column_as_df.iterrows():
            for j, other_row in column_as_df.iterrows():
                if i >= j:
                    continue
                score = SequenceMatcher(None, row, other_row).ratio()
                if score >= threshold:
                    similarity_scores[(i, j)] = score
        
        # Identify rows to remove
        rows_to_remove = []
        for (i, j), score in similarity_scores.items():
            if i not in rows_to_remove and j not in rows_to_remove:
                rows_to_remove.append(j if df_player.index[i] < df_player.index[j] else i)
        
        # Remove rows and concatenate df
        df_player = df_player.drop(rows_to_remove)
        df_complete = pd.concat([df_complete, df_player], axis=0)

        #return modified DataFrame
    return df_complete

from unidecode import unidecode
def remove_accents(text):
    return unidecode(text)

'''def del_patterns(df_line, pattern):
    lines = df_line.split('\\n')
    new_string = ''
    for line in lines:
        deleting = False
        for word in pattern:
            if deleting:
                break
            elif word in line:
                deleting = True
            else:
                deleting = False
        if not deleting:
            new_string += line + '\n'
    return new_string.strip()'''


"def del_patterns(df_line, pattern):\n    lines = df_line.split('\\n')\n    new_string = ''\n    for line in lines:\n        deleting = False\n        for word in pattern:\n            if deleting:\n                break\n            elif word in line:\n                deleting = True\n            else:\n                deleting = False\n        if not deleting:\n            new_string += line + '\n'\n    return new_string.strip()"

In [108]:
def del_patterns(df_line, pattern):
    '''
    Function which takes an input and deletes defined text pattern 
    '''
    # split up the records in lines
    lines = df_line.split('\\n')
    
    if len(lines) > 1:
        # create an empty string
        new_string = ''
        
        # iterating over the lines 
        for line in lines:
            # set deleting to False first
            deleting = False
            
            # check if any pattern word is included in the line and set deleting to True if so 
            for word in pattern:
                if word in line:
                    deleting = True
                    break
            
            # if the sentence should not be deleted, add it to the string  
            if not deleting:
                new_string += line + ' '  # Add a space after each line
            
        # remove trailing space from the new_string
        new_string = new_string.rstrip()
        
    else:
        new_string = df_line
    
    # return the string 
    return new_string


# Load Data

In [109]:
url = 'https://raw.githubusercontent.com/svisel22/SS23-BIPM-Analytics-Lab---Group-4-repository/main/data_files/all_data_v3.csv'
df = pd.read_csv(url)

In [110]:
# Filter out the Spanish data and reindex
df_en = df[df["language"] == "en"]

#Reset index
df_en = df_en.reset_index(drop=True)
df_en

,data,player,language,publishedAt
0,{'content': 'Football\nFlorian Wirtz\'s goal f...,Exequiel Palacios,en,2023-02-16T23:56:00Z
1,{'content': '[1/4]\xa0Soccer Football - Europa...,Exequiel Palacios,en,2023-02-23T20:50:50Z
2,"{'content': ""By Will Pickworth For Mailonline\...",Exequiel Palacios,en,2023-02-23T20:53:59Z
3,{'content': '\nBUENOS AIRES (AP) — World Cup w...,Exequiel Palacios,en,2023-03-03T16:40:46Z
4,{'content': 'Sign In\nSign In\nThe Star Editio...,Exequiel Palacios,en,2023-03-03T16:42:19Z
...,...,...,...,...
408,{'content': 'Real Madrid and Manchester City p...,Mykhaylo Mudryk,en,2023-05-09T20:58:09Z
409,{'content': 'Real Madrid and Manchester City p...,Mykhaylo Mudryk,en,2023-05-09T19:45:09Z
410,{'content': 'West Ham United are the sole Prem...,Mykhaylo Mudryk,en,2023-05-09T14:30:39Z
411,"{'content': ""Inter Milan beat AC Milan 2-0 in ...",Mykhaylo Mudryk,en,2023-05-09T14:17:43Z


In [111]:
print(df_en.loc[5, 'data'])

{'content': "World Cup winners Argentina will celebrate with their fans at two home games in March against Panama and Curacao in their first matches since their triumph in Qatar.\nArgentina coach Lionel Scaloni announced a 35-man squad on Friday which included Lionel Messi, Angel Di Maria and the other members of Argentina's winning squad in Qatar.\nManchester United forward Alejandro Garnacho is rewarded for impressing at Old Trafford with his first-ever callup. The 18-year-old, in his breakthrough season with Erik ten Hag's side, has contributed five goals and six assists in 30 matches this campaign.\nIn their first games since beating France in the World Cup final, Argentina will play Panama on March 23 at the Monumental Stadium in Buenos Aires. Five days later, they will host Curacao in another friendly at the northern city of Santiago del Estero.\nScaloni, 44, earlier this week extended his contract until the end of the 2026 World Cup.\nSquad:\nGoalkeepers: Emiliano Martinez (Asto

# Remove similiar rows

In [112]:
df_en = remove_similar_rows_per_player(df_en, df_en['player'].unique())
df_en

,data,player,language,publishedAt
0,{'content': 'Football\nFlorian Wirtz\'s goal f...,Exequiel Palacios,en,2023-02-16T23:56:00Z
1,{'content': '[1/4]\xa0Soccer Football - Europa...,Exequiel Palacios,en,2023-02-23T20:50:50Z
2,"{'content': ""By Will Pickworth For Mailonline\...",Exequiel Palacios,en,2023-02-23T20:53:59Z
3,{'content': '\nBUENOS AIRES (AP) — World Cup w...,Exequiel Palacios,en,2023-03-03T16:40:46Z
4,{'content': 'Sign In\nSign In\nThe Star Editio...,Exequiel Palacios,en,2023-03-03T16:42:19Z
...,...,...,...,...
24,"{'content': ""By Dominic Hogan For Mailonline\n...",Piero Hincapie,en,2023-05-16T12:22:18Z
0,{'content': 'We use cookies and other tracking...,Piero Hincapié,en,2023-04-27T04:57:02Z
1,{'content': 'Man City’s Alex Robertson makes d...,Piero Hincapié,en,2023-03-24T15:24:08Z
2,"{'content': ""\nLast updated on 19 March 202319...",Piero Hincapié,en,2023-03-19T20:03:28Z


# Transform data into lower case

In [113]:
data_lower_en = df_en.copy()

data_lower_en['data'] = data_lower_en['data'].str.lower()
data_lower_en['player'] = data_lower_en['player'].str.lower()
data_lower_en.head()

,data,player,language,publishedAt
0,{'content': 'football\nflorian wirtz\'s goal f...,exequiel palacios,en,2023-02-16T23:56:00Z
1,{'content': '[1/4]\xa0soccer football - europa...,exequiel palacios,en,2023-02-23T20:50:50Z
2,"{'content': ""by will pickworth for mailonline\...",exequiel palacios,en,2023-02-23T20:53:59Z
3,{'content': '\nbuenos aires (ap) — world cup w...,exequiel palacios,en,2023-03-03T16:40:46Z
4,{'content': 'sign in\nsign in\nthe star editio...,exequiel palacios,en,2023-03-03T16:42:19Z


# Remove patterns

In [114]:
print(data_lower_en.iloc[4].values)

["{'content': 'sign in\\nsign in\\nthe star edition\\nchange location\\nthis copy is for your personal non-commercial use only. to order\\n    presentation-ready copies of toronto star content for distribution\\n    to colleagues, clients or customers, or inquire about\\n    permissions/licensing, please go to: www.torontostarreprints.com\\nbuenos aires (ap) — world cup winner argentina will celebrate with its fans at two home games in march against panama and curacao.\\nargentina coach lionel scaloni on friday announced a 35-man squad which included lionel messi, ángel di maria and the other members of argentina’s winning squad in qatar.\\nin its first games since beating france in the world cup final, argentina will play panama on march 23 at the monumental stadium in buenos aires. five days later it will host curacao in another friendly at the northern city of santiago del estero.\\nmanchester united forward alejandro garnacho is rewarded for impressing at manchester united with his

In [115]:
data_wo_pattern_en = data_lower_en.copy()

# delete content patterns
data_wo_pattern_en['data'] = data_wo_pattern_en['data'].apply(lambda x: re.sub(r"^{\'content\': \'", "", str(x)))
data_wo_pattern_en['data'] = data_wo_pattern_en['data'].apply(lambda x: re.sub(r"{'content':", "", str(x)))

#remove patterns
patternlist_en = [
    "copyright",
    "photo",
    'image',
    "all rights reserved",
    'published by',
    'published',
    'publisher',
    'want an ad-free experience?',
    'log in',    
    'last updated on',
    'last updated', #not working yet
    'updated',
    'we use cookies and other tracking technologies',
    'we use cookies',
    'from the section',
    'filed under:',
    'when you purchase through links on our site, we may earn an affiliate commission',
    'how it works',
    'the ambury',
    'bath',
    'future publishing limited quay house',
    'to the independent?',
    'if you would prefer:',
    'want an ad-free experience?\n',
    'fourfourtwo is part of future plc, an international media group and leading digital publisher.',
    'visit our corporate site (opens in new tab).',
    ' the journal publishes the biggest breaking news in irish and international sport but for all of the 42′s insightful analysis and sharp sportswriting, subscribe\\xa0here. making a difference',
    'a mix of advertising and supporting contributions helps keep paywalls away from valuable information like this article.',
    'over 5,000 readers like you have already stepped up and support us with a monthly payment or a once-off donation.',
    'for the price of one cup of coffee each week you can make sure we can keep reliable, meaningful news open to everyone regardless of their ability to pay.',
    'support us',
    'learn more for the price of one cup of coffee each week you can make sure we can keep reliable, meaningful news open to everyone regardless of their ability to pay. to embed this post, copy the code below on your site 600px wide <iframe width="600" height="460" frameborder="0" style="border:0px;" src="https://www.thejournal.ie/https://www.thejournal.ie/ireland-france-4-6030557-mar2023/?embedpost=6030557&width=600&height=460" ></iframe> 400px wide <iframe width="600" height="460" frameborder="0" style="border:0px;" src="https://www.thejournal.ie/https://www.thejournal.ie/ireland-france-4-6030557-mar2023/?embedpost=6030557&width=400&height=460" ></iframe> 300px wide <iframe width="600" height="460" frameborder="0" style="border:0px;" src="https://www.thejournal.ie/https://www.thejournal.ie/ireland-france-4-6030557-mar2023/?embedpost=6030557&width=300&height=460" ></iframe> access to the comments facility has been disabled for this user',
    '483623.',
    'registered office: 3rd floor, latin hall, golden lane, dublin 8. please note that the journal uses cookies to improve your experience and to provide',
    'services and',
    'advertising. for more information on cookies please refer to our cookies',
    'policy. the journal supports the work of the press council of ireland and the office of the press',
    'ombudsman, and our staff operate within the code of practice. you can obtain a copy of the',
    'code, or contact the council, at www.presscouncil.ie,',
    'ph: (01) 6489130, lo-call 1890 208 080 or email: info@presscouncil.ie news images provided by press association',
    'and rollingnews.ie unless otherwise stated.',
    'unless otherwise stated. wire service provided by afp and press association. journal media does not control and is not responsible for user created content, posts, comments,',
    'submissions or preferences. users are reminded that they are fully responsible for their own',
    'and indemnify journal media in relation to such content and their ability to make such content,',
    'posts, comments and submissions available. journal media does not control and is not responsible',
    'for the content of external websites. switch to mobile site switch to desktop site',
    'create an email alert based on the current article',
    'refresh the page or navigate to another page on the site to be automatically logged inplease refresh your browser to be logged in',
    'your bookmarks in your independent premium section, under my profile',
    'referee:\\xa0artur dias (por) the journal publishes the biggest breaking news in irish and international sport but for all of the 42′s insightful analysis and sharp sportswriting, subscribe\\xa0here. making a difference',
    'learn more to embed this post, copy the code below on your site 600px wide <iframe width="600" height="460" frameborder="0" style="border:0px;" src="https://www.thejournal.ie/https://www.thejournal.ie/ireland-france-4-6030557-mar2023/?embedpost=6030557&width=600&height=460" ></iframe> 400px wide <iframe width="600" height="460" frameborder="0" style="border:0px;" src="https://www.thejournal.ie/https://www.thejournal.ie/ireland-france-4-6030557-mar2023/?embedpost=6030557&width=400&height=460" ></iframe> 300px wide <iframe width="600" height="460" frameborder="0" style="border:0px;" src="https://www.thejournal.ie/https://www.thejournal.ie/ireland-france-4-6030557-mar2023/?embedpost=6030557&width=300&height=460" ></iframe> access to the comments facility has been disabled for this user',
    'registered office: 3rd floor, latin hall, golden lane, dublin 8. please note that the journal uses cookies to improve your experience and to provide',
    'policy. the journal supports the work of the press council of ireland and the office of the press',
    'ph: (01) 6489130, lo-call 1890 208 080 or email: info@presscouncil.ie news images provided by press association',
    'unless otherwise stated. wire service provided by afp and press association. journal media does not control and is not responsible for user created content, posts, comments,',
    'created content and their own posts, comments and submissions and fully and effectively warrant',
    'for the content of external websites. switch to mobile site switch to desktop site.',
    'to bookmark your favourite articles and stories to read or reference later? start your independent premium subscription today.please refresh the page or navigate to another page on the site to be automatically logged inplease refresh your browser to be logged inlog in',
    'use cookies and other tracking technologies to improve your browsing experience on our site, show personalized content and targeted ads, analyze site traffic, and understand where our audiences come from. to learn more or opt-out, read our cookie policy. please also read our privacy notice and terms of use, which became effective december 20, 2019.',
    'by choosing i accept, you consent to our use of cookies and other tracking technologies.',
    'latest transfer news',
    'manchester city transfer news, live! latest reports, rumors, updates',
    'manchester united transfer news, live! latest reports, rumors, updates',
    'arsenal transfer news, live! latest reports, rumors, updates',
    'liverpool transfer news, rumors today, live!',
    '[ transfer news: chelsea | tottenham | man city | arsenal | man united ]',
    'we bring sports news that matters to your inbox, to help you stay informed and get a winning edge.',
    'cbs interactive',
    'cbs sports is a registered trademark of cbs broadcasting inc. commissioner.com is a registered trademark of cbs interactive inc.',
    'images by getty images and us presswire',
    'want to bookmark your favourite articles and stories to read or reference later?',
    'start your independent premium subscription today.',
    'log in want an ad-free experience?'
]



#ACTION: pattern I still can't make working:
'''
'sign in sign in',
    'sign in sign in the star edition change location',
    'this copy is for your personal non-commercial use only. to order',
    'presentation-ready copies of toronto star content for distribution    to colleagues, clients or customers, or inquire about',
    'permissions/licensing, please go to: www.torontostarreprints'
'''

"\n'sign in sign in',\n    'sign in sign in the star edition change location',\n    'this copy is for your personal non-commercial use only. to order',\n    'presentation-ready copies of toronto star content for distribution    to colleagues, clients or customers, or inquire about',\n    'permissions/licensing, please go to: www.torontostarreprints'\n"

In [116]:
data_wo_pattern_en['data'] = data_wo_pattern_en['data'].apply(lambda x: del_patterns(str(x), patternlist_en))   
data_wo_pattern_en                                                   

,data,player,language,publishedAt
0,football florian wirtz\'s goal for bayer lever...,exequiel palacios,en,2023-02-16T23:56:00Z
1,[1/4]\xa0soccer football - europa league - pla...,exequiel palacios,en,2023-02-23T20:50:50Z
2,"""by will pickworth for mailonline 5 view co...",exequiel palacios,en,2023-02-23T20:53:59Z
3,buenos aires (ap) — world cup winner argentin...,exequiel palacios,en,2023-03-03T16:40:46Z
4,sign in sign in the star edition change locati...,exequiel palacios,en,2023-03-03T16:42:19Z
...,...,...,...,...
24,"""by dominic hogan for mailonline 43 view co...",piero hincapie,en,2023-05-16T12:22:18Z
0,one for the future chelsea were rumored to hav...,piero hincapié,en,2023-04-27T04:57:02Z
1,man city’s alex robertson makes debut as aiden...,piero hincapié,en,2023-03-24T15:24:08Z
2,""" exequiel palacios scored two penalties as b...",piero hincapié,en,2023-03-19T20:03:28Z


In [117]:
#ACTION
#check whether it worked
data_affected_row = data_wo_pattern_en.copy()
filtered_rows = data_affected_row[data_affected_row['data'].str.contains('png', case=False)] #ACTION #februaryarsen #decemberwest what is that, even with the new function it's still in there

# Display the filtered rows
df_filtered = pd.DataFrame(filtered_rows['data'])
df_filtered
#print(df_filtered.iloc[10].values)

,data


# Translate emojis into words

In [118]:
data_wo_emojis = data_wo_pattern_en.copy()

data_wo_emojis['data'] = data_wo_emojis['data'].apply(lambda x: translate_emojis(str(x), language='en'))

# Reset index

In [119]:
data_wo_emojis_index = data_wo_emojis.copy()

data_wo_emojis_index.reset_index(drop=True, inplace=True)
data_wo_emojis_index

,data,player,language,publishedAt
0,football florian wirtz\'s goal for bayer lever...,exequiel palacios,en,2023-02-16T23:56:00Z
1,[1/4]\xa0soccer football - europa league - pla...,exequiel palacios,en,2023-02-23T20:50:50Z
2,"""by will pickworth for mailonline 5 view co...",exequiel palacios,en,2023-02-23T20:53:59Z
3,buenos aires (ap) — world cup winner argentin...,exequiel palacios,en,2023-03-03T16:40:46Z
4,sign in sign in the star edition change locati...,exequiel palacios,en,2023-03-03T16:42:19Z
...,...,...,...,...
389,"""by dominic hogan for mailonline 43 view co...",piero hincapie,en,2023-05-16T12:22:18Z
390,one for the future chelsea were rumored to hav...,piero hincapié,en,2023-04-27T04:57:02Z
391,man city’s alex robertson makes debut as aiden...,piero hincapié,en,2023-03-24T15:24:08Z
392,""" exequiel palacios scored two penalties as b...",piero hincapié,en,2023-03-19T20:03:28Z


# Store data in csv for Sentiment Analysis

In [120]:
# Define the folder path
folder_path = "data_clean"

# Define the file path
file_path = os.path.join(folder_path, "en_clean_1.csv")

# Save the DataFrame as a CSV file
data_wo_emojis_index.to_csv(file_path, index=False)

print("Data saved successfully.")

Data saved successfully.


# Remove noise

In [121]:
data_rm_en = data_wo_emojis.copy()

# strip_numeric -> removing digits (https://gensimr.news-r.org/reference/strip_numeric.html)
data_rm_en['data'] = data_rm_en['data'].apply(strip_numeric)

#strip links
data_rm_en['data'] = data_rm_en['data'].apply(lambda x: re.sub(r'http\S+', '', str(x)))

# strip_punctutation -> removing punctations (https://gensimr.news-r.org/reference/strip_punctuation.html)
data_rm_en['data'] = data_rm_en['data'].apply(strip_punctuation)

# strip multiple whitespaces also \n -> (https://radimrehurek.com/gensim/parsing/preprocessing.html#gensim.parsing.preprocessing.strip_multiple_whitespaces)
data_rm_en['data'] = data_rm_en['data'].apply(strip_multiple_whitespaces)

#strip spanish accents
data_rm_en['data'] = data_rm_en['data'].apply(lambda x: remove_accents(str(x)))

#strip images
#ACTION: I wouldn't do it for englisch because there is only one datapoint affected

#strip_short deletes words smaller 3
data_rm_en['data'] = data_rm_en['data'].apply(strip_short)
 

data_rm_en

,data,player,language,publishedAt
0,football florian wirtz goal for bayer leverkus...,exequiel palacios,en,2023-02-16T23:56:00Z
1,xasoccer football europa league play off secon...,exequiel palacios,en,2023-02-23T20:50:50Z
2,will pickworth for mailonline view comments ba...,exequiel palacios,en,2023-02-23T20:53:59Z
3,buenos aires world cup winner argentina will c...,exequiel palacios,en,2023-03-03T16:40:46Z
4,sign sign the star edition change location thi...,exequiel palacios,en,2023-03-03T16:42:19Z
...,...,...,...,...
24,dominic hogan for mailonline view comments tot...,piero hincapie,en,2023-05-16T12:22:18Z
0,one for the future chelsea were rumored have b...,piero hincapié,en,2023-04-27T04:57:02Z
1,man city's alex robertson makes debut aiden o'...,piero hincapié,en,2023-03-24T15:24:08Z
2,exequiel palacios scored two penalties bayer l...,piero hincapié,en,2023-03-19T20:03:28Z


# Remove Stopwords

In [122]:
# Words to keep
words_to_keep = {'don', 'didn', 'doesn', 'shouldn', 'couldn', 'wouldn', 'never', 'isn', 'cannot', 'except', 'no', 'neither', 'nor', 'cant', 'top', 'least'}

# Create modified stopwords list
modified_en_stopwords = STOPWORDS - words_to_keep

print(modified_en_stopwords)

frozenset({'should', 'across', 'computer', 'amoungst', 'being', 'hereby', 'whereby', 'beside', 'ten', 'i', 'de', 'once', 'side', 'eg', 'put', 'thru', 'is', 'several', 'detail', 'am', 'do', 'various', 'however', 'we', 'how', 'former', 'herself', 'become', 'beyond', 'its', 'thereafter', 'get', 'unless', 'perhaps', 'who', 'does', 'thence', 'forty', 'rather', 'few', 'nothing', 'what', 'your', 'whom', 'name', 'within', 'myself', 'eight', 'became', 'back', 'whole', 'against', 'by', 'through', 'did', 'fill', 'along', 'thin', 'cry', 'afterwards', 'kg', 'very', 'ie', 'at', 'couldnt', 'whereupon', 'over', 'him', 'when', 'first', 'before', 'twenty', 'co', 'found', 'third', 'with', 'hundred', 'wherever', 'if', 'make', 'last', 'becomes', 'see', 'to', 'somewhere', 'many', 'every', 'much', 'everywhere', 'serious', 'km', 'among', 'next', 'our', 'though', 'each', 'indeed', 'where', 'my', 'such', 'two', 'them', 'above', 'whereas', 'onto', 'ever', 'beforehand', 'it', 'be', 'amongst', 'were', 'you', 'or',

In [123]:
data_rm_stop_en = data_rm_en.copy()

# Apply the remove_stopwords function to the 'text' column using the apply method
data_rm_stop_en['data'] = data_rm_stop_en['data'].apply(lambda x: remove_stopwords_from_text(x, modified_en_stopwords))

# Reset index

In [124]:
data_en_pp = data_rm_stop_en.copy()

data_en_pp.reset_index(drop=True, inplace=True)

# Store data in csv for Text Clustering

In [125]:
# Define the folder path
folder_path = "data_clean"

# Define the file path
file_path = os.path.join(folder_path, "en_clean_2_1.csv")

# Save the DataFrame as a CSV file
data_en_pp.to_csv(file_path, index=False)

print("Data saved successfully.")

Data saved successfully.


# Code to check whether preprocessing worked

In [ ]:
'''Control for patterns
#check whether it worked
data_affected_row = data_wo_pattern_en.copy()
filtered_rows = data_affected_row[data_affected_row['data'].str.contains('by choosing', case=False)]

# Display the filtered rows
df_filtered = pd.DataFrame(filtered_rows['data'])
df_filtered
#print(df_filtered.iloc[0].values)
'''

"Control for patterns\n#check whether it worked\ndata_affected_row = data_wo_pattern_en.copy()\nfiltered_rows = data_affected_row[data_affected_row['data'].str.contains('by choosing', case=False)]\n\n# Display the filtered rows\ndf_filtered = pd.DataFrame(filtered_rows['data'])\ndf_filtered\n#print(df_filtered.iloc[0].values)\n"

In [ ]:
'''Control for emojis

# Unicode ranges for emojis
emoji_ranges = [
    (0x1F600, 0x1F64F),  # Emoticons
    (0x1F300, 0x1F5FF),  # Miscellaneous symbols and pictographs
    (0x1F680, 0x1F6FF),  # Transport and map symbols
    (0x2600, 0x26FF),    # Miscellaneous symbols
    (0x2700, 0x27BF),    # Dingbats
    (0xFE00, 0xFE0F),    # Variation Selectors
    (0x1F900, 0x1F9FF),  # Supplemental Symbols and Pictographs
    (0x1F1E6, 0x1F1FF)   # Flags
]

# Function to check if a character is an emoji
def is_emoji(character):
    if emoji.demojize(character) != character:
        return True
    return False

# Assuming your DataFrame is named 'df'
articles_with_untranslated_emojis = 0

# Iterate over the rows of the DataFrame
for index, row in data_wo_emojis.iterrows():
    # Counter for untranslated emojis in the current row
    untranslated_emoji_count = 0

    # Iterate over the characters in the row
    for char in str(row['data']):
        if is_emoji(char) and emoji.demojize(char) == char:
            untranslated_emoji_count += 1

    # If there is at least one untranslated emoji in the current row, increment the count of rows with untranslated emojis
    if untranslated_emoji_count > 0:
        articles_with_untranslated_emojis += 1
        print("Untranslated emojis found in row", index + 1, ":", untranslated_emoji_count)

print("Total number of rows with untranslated emojis:", articles_with_untranslated_emojis)
'''

'Control for emojis\n\n# Unicode ranges for emojis\nemoji_ranges = [\n    (0x1F600, 0x1F64F),  # Emoticons\n    (0x1F300, 0x1F5FF),  # Miscellaneous symbols and pictographs\n    (0x1F680, 0x1F6FF),  # Transport and map symbols\n    (0x2600, 0x26FF),    # Miscellaneous symbols\n    (0x2700, 0x27BF),    # Dingbats\n    (0xFE00, 0xFE0F),    # Variation Selectors\n    (0x1F900, 0x1F9FF),  # Supplemental Symbols and Pictographs\n    (0x1F1E6, 0x1F1FF)   # Flags\n]\n\n# Function to check if a character is an emoji\ndef is_emoji(character):\n    if emoji.demojize(character) != character:\n        return True\n    return False\n\n# Assuming your DataFrame is named \'df\'\narticles_with_untranslated_emojis = 0\n\n# Iterate over the rows of the DataFrame\nfor index, row in data_wo_emojis.iterrows():\n    # Counter for untranslated emojis in the current row\n    untranslated_emoji_count = 0\n\n    # Iterate over the characters in the row\n    for char in str(row[\'data\']):\n        if is_emoji(c